In [1]:
%load_ext autoreload
%autoreload 2

# XoT Tutorial

Import OpenAI model with langchain.

In [2]:
import os
from dotenv import load_dotenv

import langchain
from langchain_openai import AzureChatOpenAI

from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

langchain.debug = False  # set verbose mode to True

load_dotenv()

langchain_llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("HKUST_OPENAI_BASE"),
    api_key=os.getenv("HKUST_OPENAI_KEY"),
    api_version=os.getenv("HKUST_OPENAI_API_VERSION"),
    openai_api_type="azure",
    azure_deployment=os.getenv("HKUST_OPENAI_DEPLOYMENT_NAME"),
    verbose=True,
)

In [3]:
langchain_llm.invoke("What is LLM?")

AIMessage(content='LLM stands for Master of Laws. It is an advanced, postgraduate academic degree, pursued by those either holding an undergraduate academic law degree, a professional law degree, or an undergraduate degree in a related subject.', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 12, 'total_tokens': 55}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-f2e02f5c-06bf-4f51-a5b2-8e505a

## Direct Use of LLM

* Practice the use of LLM with langchain.
* Intent classifier: does the user want to have its tablet repaired?

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

intent_classifier_template = """
You are a customer service chatbot help to decide if the user want to repair its device. Base on the user's description, reply "Yes" if the user want to repair the device, "No" if the user do not want to repair the device, and "End" if the user want to end the conversation.

{format_instructions}

The user's input is: {user_input}
"""


class Intent(BaseModel):
    want_to_repair: str = Field(
        description="whether the user want to repair the device", example="Yes"
    )


intent_classifier_parser = JsonOutputParser(pydantic_object=Intent)

intent_classifier_prompt = PromptTemplate(
    template=intent_classifier_template,
    input_variables=["user_input"],
    partial_variables={
        "format_instructions": intent_classifier_parser.get_format_instructions()
    },
)

intent_classifier_chain = (
    intent_classifier_prompt | langchain_llm | intent_classifier_parser
)

In [5]:
intent_classifier_chain.invoke(
    {"user_input": "My ipad cannot connect to bluetooth, I'm here to get some support."}
)

{'want_to_repair': 'Yes'}

## Chain of Thoughts Prompting

* Practice the use of CoT with langchain.
* Use warranty checker as an example.

In [6]:
# Make a parser for the response
class Warranty(BaseModel):
    Analysis: str = Field(description="AI's analysis of the current situation")
    Warranty: str = Field(description="Whether the use's product have a warranty")
    Utterance: str = Field(description="AI's response to the user")

warranty_parser = JsonOutputParser(pydantic_object=Warranty)

In [7]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain_core.messages import SystemMessage

warranty_checker_examples = [
    {
        "ai_question": "When did you purchase this item?",
        "human_response": "A month ago",
        "ai_response": "Analysis: Let's think step by step. All products come with a 90-day warranty since purchase. A month is within the 90-day time span. The product must have warranty. Warranty: Yes. Utterance: Great! Your product has warranty.",
    },
    {
        "ai_question": "When did you purchase this item?",
        "human_response": "Last year.",
        "ai_response": "Analysis: Let's think step by step. All products come with a 90-day warranty. A year is more than this time span. However, we are not sure if the user purchased additional warranty. We are not sure if the product has warranty. Warranty: Unsure. Utterance: Did you purchase additional warranty?",
    },
]

warranty_checker_example_template = ChatPromptTemplate.from_messages(
    [
        ("ai", "{ai_question}"),
        ("human", "{human_response}"),
        ("ai", "{ai_response}"),
    ]
)

warranty_checker_fewshot_examples = FewShotChatMessagePromptTemplate(
    example_prompt=warranty_checker_example_template,
    examples=warranty_checker_examples,
)


warranty_checker_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful agent that help to decide if a product is out of warranty. Note that all product come with a 90-day warranty since purchase. Customer can also purchase additional warranty that extends it to 2 years. Today is May 20th, 2024."
        ),
        warranty_checker_fewshot_examples,
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                template="{user_input}\n{format_instructions}",
                input_variables=["user_input"],
                partial_variables={"format_instructions": warranty_parser.get_format_instructions()},
            )
        ),
    ]
)

warranty_checker_chain = warranty_checker_prompt | langchain_llm

In [8]:
output = warranty_checker_chain.invoke({"user_input": "I bought this item on May 1st, 2023."})
warranty_parser.invoke(output)


{'Analysis': "The product was purchased on May 1st, 2023. This means the product is now just over a year old. The standard warranty of 90 days has certainly expired, but we don't know about the extended warranty.",
 'Warranty': 'Unsure',
 'Utterance': 'Your standard 90-day warranty has expired as you purchased the product over a year ago. However, if you purchased an additional warranty that extends to 2 years, your product might still be covered. Can you please confirm if you purchased the additional warranty?'}

## Tree of thoughts

* We use tree of thoughts to practice repair plan suggestion. Users need to choose from repair, replace and trade-in options. 
* 